In [2]:
import torch
import numpy as np

In [48]:
#### LINEAR REGRESSION IN PYTORCH ####

inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [49]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [50]:
# Generate random wieghts and biases
# Every Tensor has a flag: requires_grad
# that allows for fine grained exclusion of subgraphs from gradient computation and can increase efficiency.

# This is especially useful when you want to freeze part of your model,
# or you know in advance that you’re not going to use gradients w.r.t. some parameters.
# For example if you want to finetune a pretrained CNN, it’s enough to switch the requires_grad flags in the frozen base,
# and no intermediate buffers will be saved, until the computation gets to the last layer,
# where the affine transform will use weights that require gradient, and the output of the network will also require them.

w = torch.randn(2, 3, requires_grad = True)
b = torch.randn(2, requires_grad = True)

print(w)
print(b)

tensor([[ 1.4503,  0.2345, -1.9986],
        [-0.5834, -1.7458, -1.6562]], requires_grad=True)
tensor([-1.4516, -1.2366], requires_grad=True)


In [51]:
def simple_lr_model(x):
    return x @ w.t() + b

In [52]:
preds = simple_lr_model(inputs)
print(preds)

tensor([[  34.1931, -232.0047],
        [  23.2525, -313.9467],
        [  40.2300, -381.9806],
        [  82.6160, -197.0864],
        [ -18.7702, -325.0164]], grad_fn=<AddBackward0>)


In [53]:
### LOSS FUNTION ###
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() # '*' for element wise multiplication, .numel -> # elements

In [54]:
loss = mse(targets, preds)

In [1]:
# loss.backward() computes dloss/dx for every parameter x which has requires_grad=True. These are accumulated into x.grad for every parameter x
# In pseudo-code:
# x.grad += dloss/dx

In [56]:
loss.backward() # gradient descent -> requires_grad = True

In [57]:
print(w)
print(w.grad)      

tensor([[ 1.4503,  0.2345, -1.9986],
        [-0.5834, -1.7458, -1.6562]], requires_grad=True)
tensor([[ -3183.8455,  -5236.2939,  -3096.6636],
        [-31824.7461, -35689.6602, -21830.8008]])


In [58]:
# reset values
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [61]:
with torch.no_grad():
    w -= w.grad * 10
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [62]:
w, b

(tensor([[ 1.4503,  0.2345, -1.9986],
         [-0.5834, -1.7458, -1.6562]], requires_grad=True),
 tensor([-1.4516, -1.2366], requires_grad=True))

In [73]:
# Generate predictions
preds = simple_lr_model(inputs)
print(preds)

# Calculate the loss
loss = mse(preds, targets)
print(loss)

# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

tensor([[  52.0177,  -91.0015],
        [  46.8930, -128.5029],
        [  69.2727, -161.8717],
        [  98.9082,  -57.8891],
        [   4.7772, -146.5318]], grad_fn=<AddBackward0>)
tensor(26426.8027, grad_fn=<DivBackward0>)
tensor([[ -1330.6926,  -3210.8064,  -1853.6537],
        [-17258.4180, -20013.4902, -12162.3867]])
tensor([ -21.8263, -209.1594])


In [74]:
print(w)

tensor([[ 1.5382,  0.4008, -1.9012],
        [ 0.3793, -0.6522, -0.9890]], requires_grad=True)


In [115]:
### EPOCHS ###
# learning rate => 1e-5
for i in range(2000):
    preds = simple_lr_model(inputs)
    loss = mse(targets, preds)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [116]:
print(loss)

tensor(0.6197, grad_fn=<DivBackward0>)


In [117]:
# loss from 40000 -> 0.6197

In [118]:
preds

tensor([[ 56.9609,  70.1249],
        [ 82.0368, 100.6308],
        [119.4038, 133.3403],
        [ 21.3001,  37.1386],
        [101.2111, 118.7570]], grad_fn=<AddBackward0>)

In [119]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [120]:
# noice!

In [ ]:
### USING BUILT IN FUNCTIONS ###

In [124]:
import torch.nn as nn

In [125]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [126]:
from torch.utils.data import TensorDataset

In [127]:
train_ds = TensorDataset(inputs, targets)

In [128]:
from torch.utils.data import DataLoader

In [130]:
train_ds = DataLoader(train_ds, batch_size=5, shuffle=True)

In [132]:
# for x, y in train_ds:
#     print(x)
#     print(y)

In [133]:
model = nn.Linear(3, 2)
# model.weight
# model.bias

In [134]:
preds = model(inputs)

In [137]:
import torch.nn.functional as F

In [144]:
loss = F.mse_loss(model(inputs), targets)

In [145]:
loss_fn = F.mse_loss

In [139]:
#define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [146]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [149]:
fit(1000, model, loss_fn, opt, train_ds)

Epoch [10/1000], Loss: 24.8331
Epoch [20/1000], Loss: 14.9055
Epoch [30/1000], Loss: 8.8656
Epoch [40/1000], Loss: 10.5480
Epoch [50/1000], Loss: 4.2775
Epoch [60/1000], Loss: 2.0864
Epoch [70/1000], Loss: 3.2895
Epoch [80/1000], Loss: 4.5299
Epoch [90/1000], Loss: 1.7754
Epoch [100/1000], Loss: 3.4382
Epoch [110/1000], Loss: 4.6748
Epoch [120/1000], Loss: 1.1677
Epoch [130/1000], Loss: 2.2987
Epoch [140/1000], Loss: 2.6281
Epoch [150/1000], Loss: 3.8099
Epoch [160/1000], Loss: 3.4619
Epoch [170/1000], Loss: 3.1256
Epoch [180/1000], Loss: 0.6880
Epoch [190/1000], Loss: 2.0739
Epoch [200/1000], Loss: 1.8151
Epoch [210/1000], Loss: 3.8886
Epoch [220/1000], Loss: 2.0101
Epoch [230/1000], Loss: 2.4537
Epoch [240/1000], Loss: 1.7163
Epoch [250/1000], Loss: 1.3765
Epoch [260/1000], Loss: 1.5438
Epoch [270/1000], Loss: 1.4313
Epoch [280/1000], Loss: 1.3403
Epoch [290/1000], Loss: 0.5866
Epoch [300/1000], Loss: 0.7656
Epoch [310/1000], Loss: 1.4052
Epoch [320/1000], Loss: 0.9619
Epoch [330/100

In [150]:
#Noice!